In [3]:
import os 
print(os.getcwd())

path = os.path.join(os.getcwd(), 'config')
fs = [f for f in os.listdir(path) if 'json' in f]
fs

/Users/ginazhou/Documents/GitHub/cards


['prod transfer cbi fct f24.json',
 'student.json',
 'sbx points.json',
 'welcome.json',
 'aero fx.json',
 'unique ac bistro.json',
 'aero 40 yr engagement.json',
 'f25 winter btv.json',
 'aero mobile wallet.json',
 'mbna estmt.json',
 'unique ac target.json',
 'aero 40 yr acquisition.json',
 'mbna spend stim.json',
 'f25 winter aero.json',
 'n2c.json',
 'amazon grocery spend stim.json',
 'aero 10 yr contest.json']

In [6]:
!pip install pandas


zsh:1: command not found: pip


In [22]:
import pandas as pd
df = pd.read_csv('tmp_pmdef_cnt.csv')
df.columns

Index(['campaign', 'pmdef '], dtype='object')

In [25]:
list = ['student', 'n2c', 'mbna estmt']
df.loc[df['campaign'].isin(list)].to_dict(orient='records') 

[{'campaign': 'student', 'pmdef ': 3},
 {'campaign': 'mbna estmt', 'pmdef ': 6},
 {'campaign': 'n2c', 'pmdef ': 1}]

In [ ]:
def calc_jaccard(new_json, old_jsons,n) -> list:

    def jaccard(json1, json2) -> float:
        # get key & values
        set1 = {key for key, value in json1.items() if value is True}
        set2 = {key for key, value in json2.items() if value is True}
        
        # calc intersection and union
        intersection = set1.intersection(set2)
        union = set1.union(set2)
        
        # calc jccard similarity 
        if not union:  # Avoid division by zero
            return 0.0
        return len(intersection) / len(union)

    # repeat for every json
    scored_jsons = [(old_json['title'], jaccard(new_json, old_json)) for old_json in old_jsons]
    # sort in descending order  
    scored_jsons.sort(key=lambda x: x[1], reverse=True)



    # Return the top N similar JSONs with their scores
    scored_jsons = [{"title": old_json['title'], "score": jaccard(new_json, old_json)} for old_json in old_jsons]
    # sort a list of json, descending
    sorted_jsons = sorted(scored_jsons, key=lambda x: x['score'], reverse=True)

    
    # Return the top N similar JSONs with their scores  
    return sorted_jsons[:n]     #scored_jsons[:n]


     


In [50]:
import json

new_json = json.load(open(path+'/'+'bluejays.json', 'r')) 
old_jsons = [json.load(open(path+'/'+f, 'r')) for f in fs] 

# top 3 similar offer are - jaccard: 
top3 = calc_jaccard(new_json, old_jsons, 3)

#print(f"the top 3 matches are: ")
#for idx, (json_obj, score) in enumerate(top3, start=1):
#    print(f"{idx}. {json_obj['title']}, w/ a similarity score of {score:.2f}")

top3

[{'title': 'Aeroplan FX Offer', 'score': 0.25},
 {'title': 'prod transfer - Cash Back Visa Infinite* & First Class Travel® Visa Infinite* Offers',
  'score': 0.0},
 {'title': 'Cash Back Visa - Student Offer', 'score': 0.0}]

In [3]:
def calc_hamming(new_json, old_jsons,n):

    def hamming(json1, json2):
        # get key & values
        set1 = {key for key, value in json1.items() if value is True}
        set2 = {key for key, value in json2.items() if value is True}
        
        # calc intersection and union
        keys = set1.union(set2)
        
        # Calculate Hamming distance
        hamming = sum(json1.get(key, False) != json2.get(key, False) for key in keys)
        
        total_keys = len(keys)
        return 1 - (hamming / total_keys)

    # repeat for every json
    scored_jsons = [(old_json, hamming(new_json, old_json)) for old_json in old_jsons]

    # sort in descending order  
    scored_jsons.sort(key=lambda x: x[1], reverse=True)

    # Return the top N similar JSONs with their scores
    return scored_jsons[:n]


In [32]:
top3

[({'title': 'Aeroplan FX Offer',
   'is_contest': False,
   'is_tiered_offer': False,
   'is_mobile_wallet_related': False,
   'has_annual_fee': False,
   'requires_physical_location': False,
   'includes_amazon_cards': False,
   'includes_aero_cards': True,
   'is_location_based_promotion': True,
   'points_or_cashback_or_both': 'points',
   'consumer_or_business_or_both': 'consumer',
   'payout_structure': 'one-off',
   'campaign_duration': 'months',
   'requires_opt_in': True,
   'external_partners': ['Air Canada'],
   'distribution_channel': 'launched within cards',
   'qualification_transaction_count': 'single',
   'qualification_basis': 'amount spend',
   'single_or_multiple_products': 'single',
   'td_or_mbna_products': 'TD',
   'td_rewards_or_aero_or_both': 'aero',
   'new_accounts_or_transfers_or_both': 'existing accounts'},
  0.25),
 ({'title': 'TD Cash Back Visa Infinite* and TD First Class Travel® Visa Infinite* Offers',
   'is_contest': False,
   'is_tiered_offer': True,
 

In [ ]:

# try hamming 
top3 = calc_hamming(new_json, old_jsons, 10)
print(f"")
print(f"the top 3 campaigns matching {fs[1]} are: ")
for idx, (json_obj, score) in enumerate(top3, start=1):
    print(f"{idx}. {json_obj['title']} with a score of {score:.2f}")

In [8]:
import json
from sklearn.feature_extraction.text import DictVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from statistics import mean

def calc_cosine(new_json, old_jsons, n):
    all_jsons = old_jsons + [new_json]
    
    # vectorize json 
    vectorizer = DictVectorizer(sparse=False)
    feature_vectors = vectorizer.fit_transform(all_jsons)
    
    # calc consine similarity
    new_vector = feature_vectors[-1].reshape(1, -1)   
    old_vectors = feature_vectors[:-1]   

    score = cosine_similarity(new_vector, old_vectors).flatten()
    avg = mean(scores)
    
    return scores.tolist(), avg

scores, avg_score = calculate_similarity_score(new_json, old_jsons)

print("Similarity Scores:", scores)
print("Average Similarity Score:", avg_score)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
#example 

risk_scores = [
    {'id': 1, 'score': 50, 'reasons': ['Age below 25', 'Income below $30,000']},
    {'id': 2, 'score': 30, 'reasons': ['Good credit history']}
]


pmdef_count = { 'aero mobile wallet tiered': 5 
                'aero rec & enter tiered': 1
                'aero fx': 11 
                'aero rando': 4
                'new campaign': ""}



SyntaxError: invalid syntax (2113981774.py, line 10)